In [1]:
from pathlib import Path

import pandas as pd

In [2]:
dates_to_visualize = [
    "2019-02",
    "2020-02",
    "2021-02",
    "2022-02",
    "2023-02",
    # "2024-02",
]

In [3]:
available_data_files = list(Path("../data/httparchive_metrics/nel_domain_resource_monitoring_stats").glob("*.parquet"))

used_data_files = list(filter(lambda file: file.stem in dates_to_visualize, available_data_files))
used_data_files

[WindowsPath('../data/httparchive_metrics/nel_domain_resource_monitoring_stats/2019-02.parquet'),
 WindowsPath('../data/httparchive_metrics/nel_domain_resource_monitoring_stats/2020-02.parquet'),
 WindowsPath('../data/httparchive_metrics/nel_domain_resource_monitoring_stats/2021-02.parquet'),
 WindowsPath('../data/httparchive_metrics/nel_domain_resource_monitoring_stats/2022-02.parquet'),
 WindowsPath('../data/httparchive_metrics/nel_domain_resource_monitoring_stats/2023-02.parquet')]

### Aggregate result to visualize

In [9]:
from results.result_utils import date_to_text_format, concat_data_from_files

result = concat_data_from_files(used_data_files)

result['date_formatted'] = result['date'].map(date_to_text_format)

result

2785923

### NEL Domain monitored resources ratio distribution

In [5]:
from results.result_utils import get_first_or_0

distribution_result = pd.DataFrame({}, index=[
    '(0%-10%>',
    '(10%-25%>',
    '(25%-50%>',
    '(50%-75%>',
    '(75%-100%)',
    '<100%>',
])

for month in dates_to_visualize:
    current_month_data = result[result['date'] == month].copy()
    
    month_data_col = []

    # month_result['(0%-10%>']    
    next_val = current_month_data[(current_month_data['url_domain_monitored_resources_ratio'] > 0.0) & (current_month_data['url_domain_monitored_resources_ratio'] <= 10.0)].count()
    month_data_col.append(get_first_or_0(next_val))

    # month_result['(10%-25%>']    
    next_val = current_month_data[(current_month_data['url_domain_monitored_resources_ratio'] > 10.0) & (current_month_data['url_domain_monitored_resources_ratio'] <= 25.0)].count()
    month_data_col.append(get_first_or_0(next_val))

    # month_result['(25%-50%>']    
    next_val = current_month_data[(current_month_data['url_domain_monitored_resources_ratio'] > 25.0) & (current_month_data['url_domain_monitored_resources_ratio'] <= 50.0)].count()
    month_data_col.append(get_first_or_0(next_val))

    # month_result['(50%-75%>']    
    next_val = current_month_data[(current_month_data['url_domain_monitored_resources_ratio'] > 50.0) & (current_month_data['url_domain_monitored_resources_ratio'] <= 75.0)].count()
    month_data_col.append(get_first_or_0(next_val))

    # month_result['(75%-100%)']    
    next_val = current_month_data[(current_month_data['url_domain_monitored_resources_ratio'] > 75.0) & (current_month_data['url_domain_monitored_resources_ratio'] < 100.0)].count()
    month_data_col.append(get_first_or_0(next_val))
    
    # month_result['<100%>']
    next_val = current_month_data[current_month_data['url_domain_monitored_resources_ratio'] == 100.0].count()
    month_data_col.append(get_first_or_0(next_val))
    
    
    distribution_result[date_to_text_format(month)] = month_data_col
    
distribution_result

,Feb 2019,Feb 2020,Feb 2021,Feb 2022,Feb 2023
(0%-10%>,29,188,1324,19856,23214
(10%-25%>,7,250,1074,4717,2218
(25%-50%>,8,11858,23114,5214,5341
(50%-75%>,25,10461,23678,5207,7006
(75%-100%),29,5378,32986,64864,93968
<100%>,271,81467,925923,872300,1845611
